In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle("data_sample_len2_sample_step2.pkl")

In [3]:
df.head(2)

,apen,fmean,fmed,fmode,intrange,mav,mavfd,mavsd,peak,rms,sampen,srange,std,var,zc,Emotion,Muscle,Person,Sample idx
0,1.307227,0.238228,0.253906,11.391353+7.735965j,0.296268,0.184651,0.253215,0.396857,1.000022,0.233531,1.939744,1.718309,0.233519,0.054531,238,happy,Zygomaticus Major,alar,0
1,1.326351,0.244564,0.261719,14.179856+12.743834j,0.356517,0.205449,0.280853,0.439035,0.704597,0.257154,2.101206,1.537603,0.257139,0.066121,242,happy,Zygomaticus Major,alar,1


In [4]:
df["Muscle"] = df["Muscle"].astype('category')
df["Emotion"] = df["Emotion"].astype('category')
df = df.drop(["Person", "Sample idx"], axis=1)

X = pd.get_dummies(df.drop(["Emotion", "fmode"], axis=1))
y = df["Emotion"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [6]:
from sklearn.svm import SVC

clf = SVC(kernel="rbf")
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.19230769230769232


In [7]:
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.38461538461538464


In [8]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.3076923076923077


In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50, random_state=5)
clf = clf.fit(X_train, y_train)
np.mean(clf.predict(X_test) == y_test)

0.6153846153846154

In [10]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=len(y), random_state=0, shuffle=True)
accuracies = []

for train_index, test_index in kf.split(X, y):
    clf = RandomForestClassifier(n_estimators=100, random_state=0)
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    acc = np.mean(clf.predict(X_test) == y_test)
    accuracies.append(acc)

In [11]:
np.mean(accuracies)

0.5277777777777778

In [12]:
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold

rf = RandomForestClassifier(n_estimators=100)
seed = 0
param_grid = {"min_samples_split":[2,4,8,16,32] }

# Inner CV: for hyperparameter tuning inside the leave-one-out CV
inner_cv = KFold(n_splits=10, shuffle=True, random_state=seed)

# Outer CV: leave-one-out CV
outer_cv = KFold(n_splits=len(y), shuffle=True, random_state=seed)

# Non-nested parameter search: just leave-one-out CV
clf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=inner_cv)
clf.fit(X, y)
non_nested_score = clf.best_score_

# Nested CV
clf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=inner_cv)
nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv).mean()

In [13]:
non_nested_score

0.528

In [14]:
nested_score

0.5119047619047619

In [15]:
len(y)

252